### coupled oscillators

In [12]:
import numpy as np
from qutip import Qobj, tensor, qeye, destroy, create, commutator, basis
import warnings

# Suppress warnings from QuTiP about sparse matrix efficiency, which are not critical for this example.
warnings.filterwarnings("ignore", category=UserWarning, module='qutip.fastsparse')

def effective_hamiltonian_sw(h0, h1):
    """
    Calculates the second-order effective Hamiltonian using the Schrieffer-Wolff (SW) transformation.

    The SW transformation finds an effective Hamiltonian for a low-energy subspace
    by eliminating the first-order coupling between the low- and high-energy subspaces.
    The second-order effective Hamiltonian is given by:
    H_eff = H_0 + (1/2) * [S, H_1]
    where S is the generator of the transformation, satisfying [S, H_0] = H_1.

    Args:
        h0 (Qobj): The unperturbed, diagonal part of the Hamiltonian. Its eigenstates
                   define the different energy subspaces.
        h1 (Qobj): The perturbation (interaction) part of the Hamiltonian that
                   couples the subspaces.

    Returns:
        Qobj: The effective Hamiltonian, accurate to second order in the perturbation.
    """
    print("Performing Schrieffer-Wolff transformation...")

    # Step 1: Find the eigenvalues and eigenvectors of the unperturbed Hamiltonian H0.
    # These define the energy basis in which the transformation is most easily calculated.
    evals, evecs = h0.eigenstates()

    # Step 2: Construct the SW generator S. The matrix elements of S in the eigenbasis of H0
    # are given by S_mn = <m|H1|n> / (E_m - E_n), where E_m and E_n are the eigenvalues.
    s_matrix = np.zeros_like(h0.full(), dtype=complex)
    
    # Transform the interaction Hamiltonian H1 into the eigenbasis of H0.
    h1_eb = h1.transform(evecs)

    # Get the number of states from the shape of the hamiltonian
    num_states = h0.shape[0]

    for m in range(num_states):
        for n in range(num_states):
            # The generator S should only connect states with different unperturbed energies.
            energy_diff = evals[m] - evals[n]
            if abs(energy_diff) > 1e-9:  # Avoid division by zero for degenerate states
                s_matrix[m, n] = h1_eb[m, n] / energy_diff

    # Create the generator Qobj in the eigenbasis and transform it back to the original basis.
    s_eb = Qobj(s_matrix, dims=h0.dims)
    s_op = s_eb.transform(evecs, True)

    # Step 3: Calculate the second-order effective Hamiltonian.
    # The commutator gives the second-order correction.
    h_eff = h0 + 0.5 * commutator(s_op, h1)
    
    print("Transformation complete.")
    return h_eff


"""
Main function to set up the system of two transmons coupled via a
coplanar waveguide resonator and calculate the effective Hamiltonian.
"""
# --- 1. System Parameters ---
# Define the dimensions of the Hilbert spaces for the two qubits (transmons)
# and the resonator (coplanar waveguide). We truncate the resonator's
# Hilbert space to a few levels, which is sufficient for the dispersive regime.
nq1 = 2  # Dimension for qubit 1 (0, 1)
nq2 = 2  # Dimension for qubit 2 (0, 1)
nc = 3   # Dimension for the resonator (0, 1, 2 photons)

# Define system frequencies and coupling strengths (in GHz).
# These are chosen to be in the dispersive regime, where |w_q - w_r| >> g.
w1 = 5.0  # Qubit 1 frequency
w2 = 5.2  # Qubit 2 frequency
wr = 7.5  # Resonator frequency
g1 = 0.1  # Qubit 1-resonator coupling strength
g2 = 0.1  # Qubit 2-resonator coupling strength

# --- 2. Define System Operators ---
# Create identity operators for each subsystem.
id_q1 = qeye(nq1)
id_q2 = qeye(nq2)
id_c = qeye(nc)

# Create annihilation operators for each subsystem.
a1 = destroy(nq1)
a2 = destroy(nq2)
ac = destroy(nc)

# Construct composite operators for the full Hilbert space (Q1 ⊗ Q2 ⊗ Resonator).
# These operators act on the combined system.
# Qubit 1 operators:
b1 = tensor(a1, id_q2, id_c)
b1_dag = tensor(a1.dag(), id_q2, id_c)

# Qubit 2 operators:
b2 = tensor(id_q1, a2, id_c)
b2_dag = tensor(id_q1, a2.dag(), id_c)

# Resonator operators:
c0 = tensor(id_q1, id_q2, ac)
c0_dag = tensor(id_q1, id_q2, ac.dag())

# --- 3. Construct the Full Hamiltonian ---
# The Hamiltonian is separated into two parts:
# H0: The unperturbed part, containing the individual energies of the qubits and resonator.
# H1: The interaction part, describing the coupling between the qubits and the resonator.

# Unperturbed Hamiltonian (diagonal in the number basis)
H0 = w1 * b1_dag * b1 + w2 * b2_dag * b2 + wr * c0_dag * c0

# Interaction Hamiltonian (Jaynes-Cummings type)
# This describes the exchange of excitations between each qubit and the resonator.
H1 = g1 * (b1_dag * c0 + b1 * c0_dag) + g2 * (b2_dag * c0 + b2 * c0_dag)

H_full = H0 + H1

print("--- System Definition ---")
print(f"Qubit 1 Freq (w1): {w1} GHz")
print(f"Qubit 2 Freq (w2): {w2} GHz")
print(f"Resonator Freq (wr): {wr} GHz")
print(f"Coupling g1: {g1} GHz, g2: {g2} GHz\n")

print("Full Hamiltonian (H_full):")
print(H_full)
print("\n--------------------------------\n")

# --- 4. Calculate and Project the Effective Hamiltonian ---
# Perform the Schrieffer-Wolff transformation to get the effective Hamiltonian
# for the entire system.
H_eff_full = effective_hamiltonian_sw(H0, H1)

# Project the effective Hamiltonian onto the zero-photon subspace of the resonator.
# This gives the effective Hamiltonian for the two-qubit system.
# The basis states in QuTiP are ordered such that the resonator states are the
# fastest-changing index in the tensor product representation. However, the standard
# construction `tensor(q1, q2, res)` makes the resonator the last subsystem.
# We need to find the indices corresponding to the |q1, q2, 0> states.

# Get the qubit subspace dimension
dim_qubit = nq1 * nq2
H_eff_q_subspace_data = np.zeros((dim_qubit, dim_qubit), dtype=complex)

# Basis states for the qubit subspace |ij>
qubit_basis = [tensor(basis(nq1, i), basis(nq2, j)) for i in range(nq1) for j in range(nq2)]

for i, bra_q in enumerate(qubit_basis):
    for j, ket_q in enumerate(qubit_basis):
        # Create full-space basis vectors |ij0> = |i>_q1 ⊗ |j>_q2 ⊗ |0>_c
        bra_full = tensor(bra_q, basis(nc, 0))
        ket_full = tensor(ket_q, basis(nc, 0))
        
        # Calculate the matrix element. The result of this operation is a scalar (complex number).
        # The .data attribute is not needed.
        H_eff_q_subspace_data[i, j] = (bra_full.dag() * H_eff_full * ket_full)

H_eff_q_subspace = Qobj(H_eff_q_subspace_data, dims=[[nq1, nq2], [nq1, nq2]])

print("\n--- Results ---")
print("Effective Hamiltonian in the two-qubit (zero-photon) subspace (H_eff_q_subspace):")
print(np.round(H_eff_q_subspace.full(), 5))

# The resulting matrix contains:
# - Diagonal elements: Renormalized qubit frequencies (Lamb shift).
# - Off-diagonal elements: Effective qubit-qubit (XX) coupling, g_eff.

# --- 5. Further Diagonalization to find ZZ coupling ---
# The effective qubit Hamiltonian H_eff_q_subspace still has off-diagonal
# XX-coupling terms. We can perform another SW transformation on this smaller
# Hamiltonian to find the ZZ coupling strength.

print("\nPerforming second SW transformation on the qubit subspace to find ZZ coupling...")
H0_q = Qobj(np.diag(np.diag(H_eff_q_subspace.full())), dims=H_eff_q_subspace.dims)
H1_q = H_eff_q_subspace - H0_q

H_eff_diag = effective_hamiltonian_sw(H0_q, H1_q)

print("\nFinal Diagonal Effective Hamiltonian (H_eff_diag):")
print(np.round(H_eff_diag.full(), 5))

# The diagonal elements of this final matrix represent the fully renormalized
# energy levels of the two-qubit system. The ZZ coupling strength (zeta)
# can be extracted from these eigenvalues:
# zeta = (E_11 - E_10) - (E_01 - E_00)
E00 = H_eff_diag[0, 0] # Corresponds to state |00>
E01 = H_eff_diag[1, 1] # Corresponds to state |01>
E10 = H_eff_diag[2, 2] # Corresponds to state |10>
E11 = H_eff_diag[3, 3] # Corresponds to state |11>

zeta = (E11 - E10) - (E01 - E00)

# The effective XX coupling g_eff can be read from H_eff_q_subspace
g_eff = H_eff_q_subspace[1, 2]

print("\n--- Extracted Parameters ---")
print(f"Effective XX coupling (g_eff) \u2248 {g_eff.real:.5f} GHz")
print(f"Effective ZZ coupling (zeta) \u2248 {zeta.real * 1000:.3f} MHz")



--- System Definition ---
Qubit 1 Freq (w1): 5.0 GHz
Qubit 2 Freq (w2): 5.2 GHz
Resonator Freq (wr): 7.5 GHz
Coupling g1: 0.1 GHz, g2: 0.1 GHz

Full Hamiltonian (H_full):
Quantum object: dims=[[2, 2, 3], [2, 2, 3]], shape=(12, 12), type='oper', dtype=Dia, isherm=True
Qobj data =
[[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.        ]
 [ 0.          7.5         0.          0.1         0.          0.
   0.1         0.          0.          0.          0.          0.        ]
 [ 0.          0.         15.          0.          0.14142136  0.
   0.          0.14142136  0.          0.          0.          0.        ]
 [ 0.          0.1         0.          5.2         0.          0.
   0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.14142136  0.         12.7         0.
   0.          0.          0.          0.1         0.          0.        ]
 [ 0.          

## coupled qubits

In [15]:
import numpy as np
from qutip import Qobj, tensor, qeye, destroy, create, commutator, basis
import warnings

# Suppress warnings from QuTiP about sparse matrix efficiency, which are not critical for this example.
warnings.filterwarnings("ignore", category=UserWarning, module='qutip.fastsparse')

def effective_hamiltonian_sw(h0, h1):
    """
    Calculates the second-order effective Hamiltonian using the Schrieffer-Wolff (SW) transformation.

    The SW transformation finds an effective Hamiltonian for a low-energy subspace
    by eliminating the first-order coupling between the low- and high-energy subspaces.
    The second-order effective Hamiltonian is given by:
    H_eff = H_0 + (1/2) * [S, H_1]
    where S is the generator of the transformation, satisfying [S, H_0] = H_1.

    Args:
        h0 (Qobj): The unperturbed, diagonal part of the Hamiltonian. Its eigenstates
                   define the different energy subspaces.
        h1 (Qobj): The perturbation (interaction) part of the Hamiltonian that
                   couples the subspaces.

    Returns:
        Qobj: The effective Hamiltonian, accurate to second order in the perturbation.
    """
    print("Performing Schrieffer-Wolff transformation...")

    # Step 1: Find the eigenvalues and eigenvectors of the unperturbed Hamiltonian H0.
    # These define the energy basis in which the transformation is most easily calculated.
    evals, evecs = h0.eigenstates()

    # Step 2: Construct the SW generator S. The matrix elements of S in the eigenbasis of H0
    # are given by S_mn = <m|H1|n> / (E_m - E_n), where E_m and E_n are the eigenvalues.
    s_matrix = np.zeros_like(h0.full(), dtype=complex)
    
    # Transform the interaction Hamiltonian H1 into the eigenbasis of H0.
    h1_eb = h1.transform(evecs)

    # Get the number of states from the shape of the hamiltonian
    num_states = h0.shape[0]

    for m in range(num_states):
        for n in range(num_states):
            # The generator S should only connect states with different unperturbed energies.
            energy_diff = evals[m] - evals[n]
            if abs(energy_diff) > 1e-9:  # Avoid division by zero for degenerate states
                s_matrix[m, n] = h1_eb[m, n] / energy_diff

    # Create the generator Qobj in the eigenbasis and transform it back to the original basis.
    s_eb = Qobj(s_matrix, dims=h0.dims)
    s_op = s_eb.transform(evecs, True)

    # Step 3: Calculate the second-order effective Hamiltonian.
    # The commutator gives the second-order correction.
    h_eff = h0 + 0.5 * commutator(s_op, h1)
    
    print("Transformation complete.")
    return h_eff


"""
Main function to set up the system of two transmons (anharmonic oscillators)
coupled via a coplanar waveguide resonator and calculate the effective Hamiltonian.
"""
# --- 1. System Parameters ---
# To model transmons, we use a larger Hilbert space (nq=3) to include the |2> state,
# which is essential for capturing the effects of anharmonicity.
nq1 = 3  # Dimension for transmon 1 (0, 1, 2 levels)
nq2 = 3  # Dimension for transmon 2 (0, 1, 2 levels)
nc = 3   # Dimension for the resonator (0, 1, 2 photons)

# Define system frequencies and coupling strengths (in GHz).
w1 = 5.0    # Transmon 1 frequency (w01)
w2 = 5.2    # Transmon 2 frequency (w01)
wr = 7.5    # Resonator frequency
g1 = 0.1    # Transmon 1-resonator coupling strength
g2 = 0.1    # Transmon 2-resonator coupling strength

# NEW: Define anharmonicity for each transmon (alpha = w12 - w01)
alpha1 = -0.25 # Anharmonicity of transmon 1 (in GHz)
alpha2 = -0.2 # Anharmonicity of transmon 2 (in GHz)

# --- 2. Define System Operators ---
# Create identity operators for each subsystem.
id_q1 = qeye(nq1)
id_q2 = qeye(nq2)
id_c = qeye(nc)

# Create annihilation operators for each subsystem in their respective Hilbert spaces.
a1 = destroy(nq1)
a2 = destroy(nq2)
ac = destroy(nc)

# Construct composite operators for the full Hilbert space (Q1 ⊗ Q2 ⊗ Resonator).
# Transmon 1 operators:
b1 = tensor(a1, id_q2, id_c)
b1_dag = tensor(a1.dag(), id_q2, id_c)

# Transmon 2 operators:
b2 = tensor(id_q1, a2, id_c)
b2_dag = tensor(id_q1, a2.dag(), id_c)

# Resonator operators:
c0 = tensor(id_q1, id_q2, ac)
c0_dag = tensor(id_q1, id_q2, ac.dag())

# --- 3. Construct the Full Hamiltonian ---
# H0: The unperturbed part, including the individual transmon and resonator energies,
#     PLUS the crucial anharmonicity term for each transmon.
# H1: The interaction part (Jaynes-Cummings type coupling).

# Anharmonicity terms for the transmons
H_anharmonic_1 = (alpha1 / 2) * b1_dag * b1_dag * b1 * b1
H_anharmonic_2 = (alpha2 / 2) * b2_dag * b2_dag * b2 * b2

# Unperturbed Hamiltonian (diagonal in the number basis)
H0 = (w1 * b1_dag * b1) + (w2 * b2_dag * b2) + (wr * c0_dag * c0) + H_anharmonic_1 + H_anharmonic_2

# Interaction Hamiltonian
H1 = g1 * (b1_dag * c0 + b1 * c0_dag) + g2 * (b2_dag * c0 + b2 * c0_dag)

H_full = H0 + H1

print("--- System Definition (with Anharmonicity) ---")
print(f"Transmon 1 Freq (w1): {w1} GHz, Anharmonicity (a1): {alpha1} GHz")
print(f"Transmon 2 Freq (w2): {w2} GHz, Anharmonicity (a2): {alpha2} GHz")
print(f"Resonator Freq (wr): {wr} GHz")
print(f"Coupling g1: {g1} GHz, g2: {g2} GHz\n")

# --- 4. Calculate and Project the Effective Hamiltonian ---
H_eff_full = effective_hamiltonian_sw(H0, H1)

# Project the effective Hamiltonian onto the computational subspace, which is the
# { |00>, |01>, |10>, |11> } manifold within the zero-photon subspace of the resonator.

dim_qubit_subspace = 2 * 2
H_eff_q_subspace_data = np.zeros((dim_qubit_subspace, dim_qubit_subspace), dtype=complex)

# Create a list of basis vectors for the computational subspace { |00>, |01>, |10>, |11> }
# These are created from the larger transmon Hilbert spaces (nq1=3, nq2=3).
qubit_comp_basis = [tensor(basis(nq1, i), basis(nq2, j)) for i in range(2) for j in range(2)]

for i, bra_q_comp in enumerate(qubit_comp_basis):
    for j, ket_q_comp in enumerate(qubit_comp_basis):
        # Create full-space basis vectors by tensoring with the resonator ground state |0>_c
        bra_full = tensor(bra_q_comp, basis(nc, 0))
        ket_full = tensor(ket_q_comp, basis(nc, 0))
        
        # Calculate the matrix element.
        H_eff_q_subspace_data[i, j] = (bra_full.dag() * H_eff_full * ket_full)

# Create the final 4x4 effective qubit Hamiltonian
H_eff_q_subspace = Qobj(H_eff_q_subspace_data, dims=[[2, 2], [2, 2]])

print("\n--- Results ---")
print("Effective Hamiltonian in the two-qubit (zero-photon) subspace (H_eff_q_subspace):")
print(np.round(H_eff_q_subspace.full(), 5))

# --- 5. Further Diagonalization to find ZZ coupling ---
print("\nPerforming second SW transformation on the qubit subspace to find ZZ coupling...")
H0_q = Qobj(np.diag(np.diag(H_eff_q_subspace.full())), dims=H_eff_q_subspace.dims)
H1_q = H_eff_q_subspace - H0_q

H_eff_diag = effective_hamiltonian_sw(H0_q, H1_q)

print("\nFinal Diagonal Effective Hamiltonian (H_eff_diag):")
print(np.round(H_eff_diag.full(), 5))

# The diagonal elements of this final matrix represent the fully renormalized
# energy levels of the two-qubit system. The ZZ coupling strength (zeta)
# can be extracted from these eigenvalues:
# zeta = (E_11 - E_10) - (E_01 - E_00)
E00 = H_eff_diag[0, 0] # Corresponds to state |00>
E01 = H_eff_diag[1, 1] # Corresponds to state |01>
E10 = H_eff_diag[2, 2] # Corresponds to state |10>
E11 = H_eff_diag[3, 3] # Corresponds to state |11>

zeta = (E11 - E10) - (E01 - E00)

# The effective XX coupling g_eff can be read from H_eff_q_subspace
g_eff = H_eff_q_subspace[1, 2]

print("\n--- Extracted Parameters ---")
print(f"Effective XX coupling (g_eff) \u2248 {g_eff.real:.5f} GHz")
print(f"Effective ZZ coupling (zeta) \u2248 {zeta.real * 1000:.3f} MHz")




--- System Definition (with Anharmonicity) ---
Transmon 1 Freq (w1): 5.0 GHz, Anharmonicity (a1): -0.25 GHz
Transmon 2 Freq (w2): 5.2 GHz, Anharmonicity (a2): -0.2 GHz
Resonator Freq (wr): 7.5 GHz
Coupling g1: 0.1 GHz, g2: 0.1 GHz

Performing Schrieffer-Wolff transformation...
Transformation complete.

--- Results ---
Effective Hamiltonian in the two-qubit (zero-photon) subspace (H_eff_q_subspace):
[[ 0.000000e+00+0.j  0.000000e+00+0.j  0.000000e+00+0.j  0.000000e+00+0.j]
 [ 0.000000e+00+0.j  5.195650e+00+0.j -4.170000e-03+0.j  0.000000e+00+0.j]
 [ 0.000000e+00+0.j -4.170000e-03+0.j  4.996000e+00+0.j  0.000000e+00+0.j]
 [ 0.000000e+00+0.j  0.000000e+00+0.j  0.000000e+00+0.j  1.019165e+01+0.j]]

Performing second SW transformation on the qubit subspace to find ZZ coupling...
Performing Schrieffer-Wolff transformation...
Transformation complete.

Final Diagonal Effective Hamiltonian (H_eff_diag):
[[ 0.     +0.j  0.     +0.j  0.     +0.j  0.     +0.j]
 [ 0.     +0.j  5.19574+0.j  0.     +

## coupled qubits 4th order

In [18]:
import numpy as np
import math
from qutip import Qobj, tensor, qeye, destroy, create, commutator, basis
import warnings

# Suppress warnings from QuTiP about sparse matrix efficiency, which are not critical for this example.
warnings.filterwarnings("ignore", category=UserWarning, module='qutip.fastsparse')

def schrieffer_wolff_expansion(h0, v, order=4):
    """
    Calculates the effective Hamiltonian using the Schrieffer-Wolff (SW) transformation
    up to a specified order in the perturbation V.

    This is achieved by computing the nested commutator series from the BCH expansion:
    H_eff = H_0 + H_1 + H_2 + ...
    where H_k involves k nested commutators.

    Args:
        h0 (Qobj): The unperturbed, diagonal part of the Hamiltonian.
        v (Qobj): The perturbation (interaction) part of the Hamiltonian.
        order (int): The highest order of the expansion to compute (e.g., 4 for ZZ).

    Returns:
        Qobj: The effective Hamiltonian, accurate to the specified order.
    """
    print(f"Performing Schrieffer-Wolff transformation to order {order}...")

    # Step 1: Find the eigenvalues and eigenvectors of the unperturbed Hamiltonian H0.
    evals, evecs = h0.eigenstates()

    # Step 2: Construct the SW generator S, which satisfies [S, H0] = V.
    s_matrix = np.zeros_like(h0.full(), dtype=complex)
    v_eb = v.transform(evecs) # Transform V to the eigenbasis of H0
    num_states = h0.shape[0]

    for m in range(num_states):
        for n in range(num_states):
            energy_diff = evals[m] - evals[n]
            if abs(energy_diff) > 1e-9:
                s_matrix[m, n] = v_eb[m, n] / energy_diff

    s_op = Qobj(s_matrix, dims=h0.dims).transform(evecs, True)

    # Step 3: Iteratively calculate the higher-order terms of the expansion.
    # H_eff = H0 + (1/2)[S,V] + (1/6)[S,[S,V]] + (1/24)[S,[S,[S,V]]] + ...
    h_eff = h0.copy()
    last_commutator = v
    if order >= 2:
        for k in range(2, order + 1):
            # The k-th order term is (1/k!) * [S, [S, ... [S, V]...]]
            # This can be computed recursively.
            current_commutator = commutator(s_op, last_commutator)
            h_eff += (1.0 / math.factorial(k)) * current_commutator
            last_commutator = current_commutator
    
    print("Transformation complete.")
    return h_eff


def main():
    """
    Main function to set up the system of two transmons (anharmonic oscillators)
    coupled via a coplanar waveguide resonator and calculate the effective Hamiltonian.
    """
    # --- 1. System Parameters ---
    nq1 = 3  # Dimension for transmon 1 (0, 1, 2 levels)
    nq2 = 3  # Dimension for transmon 2 (0, 1, 2 levels)
    nc = 3   # Dimension for the resonator (0, 1, 2 photons)

    # Define system frequencies and coupling strengths (in GHz).
    w1 = 5.0
    w2 = 5.2
    wr = 7.5
    g1 = 0.1
    g2 = 0.1
    
    alpha1 = -0.25
    alpha2 = -0.20

    # --- 2. Define System Operators ---
    id_q1, id_q2, id_c = qeye(nq1), qeye(nq2), qeye(nc)
    a1, a2, ac = destroy(nq1), destroy(nq2), destroy(nc)

    b1 = tensor(a1, id_q2, id_c)
    b2 = tensor(id_q1, a2, id_c)
    c0 = tensor(id_q1, id_q2, ac)

    # --- 3. Construct the Full Hamiltonian ---
    H_anharmonic_1 = (alpha1 / 2) * b1.dag() * b1.dag() * b1 * b1
    H_anharmonic_2 = (alpha2 / 2) * b2.dag() * b2.dag() * b2 * b2
    
    H0 = (w1 * b1.dag() * b1) + (w2 * b2.dag() * b2) + (wr * c0.dag() * c0) + H_anharmonic_1 + H_anharmonic_2
    H1 = g1 * (b1.dag() * c0 + b1 * c0.dag()) + g2 * (b2.dag() * c0 + b2 * c0.dag())
    
    print("--- System Definition (with Anharmonicity) ---")
    print(f"Transmon 1 Freq (w1): {w1} GHz, Anharmonicity (a1): {alpha1} GHz")
    print(f"Transmon 2 Freq (w2): {w2} GHz, Anharmonicity (a2): {alpha2} GHz")
    print(f"Resonator Freq (wr): {wr} GHz")
    print(f"Coupling g1: {g1} GHz, g2: {g2} GHz\n")
    
    # --- 4. Calculate the 4th-Order Effective Hamiltonian ---
    # We now use the new function to calculate H_eff to 4th order.
    H_eff_full = schrieffer_wolff_expansion(H0, H1, order=4)

    # --- 5. Project into the Computational Subspace ---
    # Project the full effective Hamiltonian onto the computational subspace,
    # which is the { |00>, |01>, |10>, |11> } manifold.
    dim_qubit_subspace = 4
    H_eff_q_subspace_data = np.zeros((dim_qubit_subspace, dim_qubit_subspace), dtype=complex)
    qubit_comp_basis = [tensor(basis(nq1, i), basis(nq2, j)) for i in range(2) for j in range(2)]

    for i, bra_q_comp in enumerate(qubit_comp_basis):
        for j, ket_q_comp in enumerate(qubit_comp_basis):
            bra_full = tensor(bra_q_comp, basis(nc, 0))
            ket_full = tensor(ket_q_comp, basis(nc, 0))
            H_eff_q_subspace_data[i, j] = (bra_full.dag() * H_eff_full * ket_full)

    H_eff_q_subspace = Qobj(H_eff_q_subspace_data, dims=[[2, 2], [2, 2]])

    print("\n--- Results from 4th-Order SW Transformation ---")
    print("Effective Hamiltonian in the two-qubit (zero-photon) subspace:")
    print(np.round(H_eff_q_subspace.full(), 5))
    
    # --- 6. Extract Parameters from the Effective Hamiltonian ---
    # The effective XX coupling g_eff is the off-diagonal element coupling |01> and |10>
    g_eff = H_eff_q_subspace[1, 2]

    # The ZZ coupling (zeta) can now be extracted directly from the diagonal elements
    # of the effective Hamiltonian, which represent the energy levels E_ij.
    # zeta = (E_11 - E_10) - (E_01 - E_00)
    E00 = H_eff_q_subspace[0, 0]
    E01 = H_eff_q_subspace[1, 1]
    E10 = H_eff_q_subspace[2, 2]
    E11 = H_eff_q_subspace[3, 3]
    
    zeta = (E11 - E10) - (E01 - E00)

    print("\n--- Final Extracted Parameters (from 4th-Order SW) ---")
    print(f"Effective XX coupling (g_eff) \u2248 {g_eff.real:.5f} GHz")
    print(f"Effective ZZ coupling (zeta) \u2248 {zeta.real * 1000:.3f} MHz")

if __name__ == "__main__":
    main()


--- System Definition (with Anharmonicity) ---
Transmon 1 Freq (w1): 5.0 GHz, Anharmonicity (a1): -0.25 GHz
Transmon 2 Freq (w2): 5.2 GHz, Anharmonicity (a2): -0.2 GHz
Resonator Freq (wr): 7.5 GHz
Coupling g1: 0.1 GHz, g2: 0.1 GHz

Performing Schrieffer-Wolff transformation to order 4...
Transformation complete.

--- Results from 4th-Order SW Transformation ---
Effective Hamiltonian in the two-qubit (zero-photon) subspace:
[[ 0.000000e+00+0.j  0.000000e+00+0.j  0.000000e+00+0.j  0.000000e+00+0.j]
 [ 0.000000e+00+0.j  5.195660e+00+0.j -4.170000e-03+0.j  0.000000e+00+0.j]
 [ 0.000000e+00+0.j -4.170000e-03+0.j  4.996000e+00+0.j  0.000000e+00+0.j]
 [ 0.000000e+00+0.j  0.000000e+00+0.j  0.000000e+00+0.j  1.019166e+01+0.j]]

--- Final Extracted Parameters (from 4th-Order SW) ---
Effective XX coupling (g_eff) ≈ -0.00417 GHz
Effective ZZ coupling (zeta) ≈ -0.001 MHz


In [20]:
import numpy as np
from qutip import Qobj, tensor, qeye, destroy, create, commutator, basis
import warnings

# Suppress warnings from QuTiP about sparse matrix efficiency, which are not critical for this example.
warnings.filterwarnings("ignore", category=UserWarning, module='qutip.fastsparse')

def schrieffer_wolff_expansion(h0, v, order=4):
    """
    Calculates the effective Hamiltonian using the Schrieffer-Wolff (SW) transformation
    up to a specified order in the perturbation V.

    This is achieved by computing the nested commutator series from the BCH expansion:
    H_eff = H_0 + H_1 + H_2 + ...
    where H_k involves k nested commutators.

    Args:
        h0 (Qobj): The unperturbed, diagonal part of the Hamiltonian.
        v (Qobj): The perturbation (interaction) part of the Hamiltonian.
        order (int): The highest order of the expansion to compute (e.g., 4 for ZZ).

    Returns:
        Qobj: The effective Hamiltonian, accurate to the specified order.
    """
    print(f"Performing Schrieffer-Wolff transformation to order {order}...")

    # Step 1: Find the eigenvalues and eigenvectors of the unperturbed Hamiltonian H0.
    evals, evecs = h0.eigenstates()

    # Step 2: Construct the SW generator S, which satisfies [S, H0] = V.
    s_matrix = np.zeros_like(h0.full(), dtype=complex)
    v_eb = v.transform(evecs) # Transform V to the eigenbasis of H0
    num_states = h0.shape[0]

    for m in range(num_states):
        for n in range(num_states):
            energy_diff = evals[m] - evals[n]
            if abs(energy_diff) > 1e-9:
                s_matrix[m, n] = v_eb[m, n] / energy_diff

    s_op = Qobj(s_matrix, dims=h0.dims).transform(evecs, True)

    # Step 3: Iteratively calculate the higher-order terms of the expansion.
    # H_eff = H0 + (1/2)[S,V] + (1/6)[S,[S,V]] + (1/24)[S,[S,[S,V]]] + ...
    h_eff = h0.copy()
    last_commutator = v
    if order >= 2:
        for k in range(2, order + 1):
            # The k-th order term is (1/k!) * [S, [S, ... [S, V]...]]
            # This can be computed recursively.
            current_commutator = commutator(s_op, last_commutator)
            h_eff += (1.0 / math.factorial(k)) * current_commutator
            last_commutator = current_commutator
    
    print("Transformation complete.")
    return h_eff


def main():
    """
    Main function to set up the system of two transmons (anharmonic oscillators)
    coupled via a coplanar waveguide resonator and calculate the effective Hamiltonian.
    """
    # --- 1. System Parameters ---
    nq1 = 3  # Dimension for transmon 1 (0, 1, 2 levels)
    nq2 = 3  # Dimension for transmon 2 (0, 1, 2 levels)
    nc = 3   # Dimension for the resonator (0, 1, 2 photons)

    # Define system frequencies and coupling strengths (in GHz).
    w1 = 5.0
    w2 = 5.2
    wr = 7.5
    g1 = 0.1
    g2 = 0.1
    
    alpha1 = -0.25
    alpha2 = -0.20
    # NEW: Added anharmonicity for the coupler, which is essential for ZZ.
    alpha_c = -0.30 

    # --- 2. Define System Operators ---
    id_q1, id_q2, id_c = qeye(nq1), qeye(nq2), qeye(nc)
    a1, a2, ac = destroy(nq1), destroy(nq2), destroy(nc)

    b1 = tensor(a1, id_q2, id_c)
    b2 = tensor(id_q1, a2, id_c)
    c0 = tensor(id_q1, id_q2, ac)

    # --- 3. Construct the Full Hamiltonian ---
    H_anharmonic_1 = (alpha1 / 2) * b1.dag() * b1.dag() * b1 * b1
    H_anharmonic_2 = (alpha2 / 2) * b2.dag() * b2.dag() * b2 * b2
    # NEW: Anharmonic term for the coupler.
    H_anharmonic_c = (alpha_c / 2) * c0.dag() * c0.dag() * c0 * c0
    
    H0 = (w1 * b1.dag() * b1) + (w2 * b2.dag() * b2) + (wr * c0.dag() * c0) + H_anharmonic_1 + H_anharmonic_2 + H_anharmonic_c
    H1 = g1 * (b1.dag() * c0 + b1 * c0.dag()) + g2 * (b2.dag() * c0 + b2 * c0.dag())
    
    print("--- System Definition (with Anharmonicity for All Components) ---")
    print(f"Transmon 1 Freq (w1): {w1} GHz, Anharmonicity (a1): {alpha1} GHz")
    print(f"Transmon 2 Freq (w2): {w2} GHz, Anharmonicity (a2): {alpha2} GHz")
    print(f"Coupler Freq (wr): {wr} GHz, Anharmonicity (ac): {alpha_c} GHz")
    print(f"Coupling g1: {g1} GHz, g2: {g2} GHz\n")
    
    # --- 4. Calculate the 4th-Order Effective Hamiltonian ---
    H_eff_full = schrieffer_wolff_expansion(H0, H1, order=4)

    # --- 5. Project into the Computational Subspace ---
    dim_qubit_subspace = 4
    H_eff_q_subspace_data = np.zeros((dim_qubit_subspace, dim_qubit_subspace), dtype=complex)
    qubit_comp_basis = [tensor(basis(nq1, i), basis(nq2, j)) for i in range(2) for j in range(2)]

    for i, bra_q_comp in enumerate(qubit_comp_basis):
        for j, ket_q_comp in enumerate(qubit_comp_basis):
            bra_full = tensor(bra_q_comp, basis(nc, 0))
            ket_full = tensor(ket_q_comp, basis(nc, 0))
            H_eff_q_subspace_data[i, j] = (bra_full.dag() * H_eff_full * ket_full)

    H_eff_q_subspace = Qobj(H_eff_q_subspace_data, dims=[[2, 2], [2, 2]])

    print("\n--- Results from 4th-Order SW Transformation ---")
    print("Effective Hamiltonian in the two-qubit (zero-photon) subspace:")
    print(np.round(H_eff_q_subspace.full(), 5))
    
    # --- 6. Extract Parameters from the Effective Hamiltonian ---
    g_eff = H_eff_q_subspace[1, 2]
    
    E00 = H_eff_q_subspace[0, 0]
    E01 = H_eff_q_subspace[1, 1]
    E10 = H_eff_q_subspace[2, 2]
    E11 = H_eff_q_subspace[3, 3]
    
    zeta = (E11 - E10) - (E01 - E00)

    print("\n--- Final Extracted Parameters (from 4th-Order SW) ---")
    print(f"Effective XX coupling (g_eff) \u2248 {g_eff.real:.5f} GHz")
    print(f"Effective ZZ coupling (zeta) \u2248 {zeta.real * 1000:.3f} MHz")

if __name__ == "__main__":
    main()


--- System Definition (with Anharmonicity for All Components) ---
Transmon 1 Freq (w1): 5.0 GHz, Anharmonicity (a1): -0.25 GHz
Transmon 2 Freq (w2): 5.2 GHz, Anharmonicity (a2): -0.2 GHz
Coupler Freq (wr): 7.5 GHz, Anharmonicity (ac): -0.3 GHz
Coupling g1: 0.1 GHz, g2: 0.1 GHz

Performing Schrieffer-Wolff transformation to order 4...
Transformation complete.

--- Results from 4th-Order SW Transformation ---
Effective Hamiltonian in the two-qubit (zero-photon) subspace:
[[ 0.000000e+00+0.j  0.000000e+00+0.j  0.000000e+00+0.j  0.000000e+00+0.j]
 [ 0.000000e+00+0.j  5.195660e+00+0.j -4.170000e-03+0.j  0.000000e+00+0.j]
 [ 0.000000e+00+0.j -4.170000e-03+0.j  4.996000e+00+0.j  0.000000e+00+0.j]
 [ 0.000000e+00+0.j  0.000000e+00+0.j  0.000000e+00+0.j  1.019166e+01+0.j]]

--- Final Extracted Parameters (from 4th-Order SW) ---
Effective XX coupling (g_eff) ≈ -0.00417 GHz
Effective ZZ coupling (zeta) ≈ -0.002 MHz
